In [ ]:
# Author
# Prabhu Chandhar (email: prabhu@chandhar-labs.com)

from __future__ import division
import numpy as np
import matplotlib
#matplotlib.use('Agg')
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt0
import matplotlib.pyplot as plt1
import matplotlib.pyplot as plt2
import matplotlib.pyplot as plt3
import matplotlib.pyplot as plt4
import matplotlib.pyplot as plt5
import matplotlib.pyplot as plt6
import matplotlib.pyplot as plt7

import math
import numpy as np
from numpy import *
from scipy import signal
from scipy.fftpack import fftshift
from contextlib import closing
import sys

def load_iq(filename):
    x = np.fromfile(filename, np.uint8) + np.int8(-127) #adding a signed int8 to an unsigned one results in an int16 array
    x1 =  x[::2]/128 #even numbered samples
    x2  = x[1::2]/128 #odd numbered samples
    return x1+x2*1j   #return complex samples


def avgPS( x, N, fs):
    M = len(x)//N
    x_ = reshape(x[:M*N],(M,N)) * np.hamming(N)[None,:]
    X = np.fft.fftshift(np.fft.fft(x_,axis=1),axes=1)
    return r_[-N/2.0:N/2.0]/N*fs, mean(abs(X**2),axis=0)

def myspectrogram_hann_ovlp(x, m, fs, fc,dbf = 60):
    # Plot the spectrogram of x.
    # First take the original signal x and split it into blocks of length m
    # This corresponds to using a rectangular window %    
    
    isreal_bool = isreal(x).all()
    
    # pad x up to a multiple of m 
    lx = len(x);
    nt = (lx + m - 1) // m
    x = append(x,zeros(-lx+nt*m))
    x = x.reshape((m//2,nt*2), order='F')
    x = concatenate((x,x),axis=0)
    x = x.reshape((m*nt*2,1),order='F')
    x = x[r_[m//2:len(x),ones(m//2)*(len(x)-1)].astype(int)].reshape((m,nt*2),order='F')
    xmw = x * hanning(m)[:,None]; 
    a,b = np.shape(x)
    # frequency index
    t_range = np.linspace(0.0,lx / fs,b)#[0.0, lx / fs]
##    x = np.linspace(.0001,4,1000)
    f_range = np.linspace(-fs / 2.0 + fc,fs / 2.0 + fc,a)# [-fs / 2.0 + fc, fs / 2.0 + fc]
    return t_range, f_range, xmf




In [ ]:
f_c = 940.4e6#sys.argv[1]#938.8e6
sample_rate = 2.4e6
iq_samples = load_iq('../input/datfiles/Samples_for_Prediction/Samples_for_Prediction/GSM_935.4.dat')

print("Processing the samples ...")

iq_samples = iq_samples[2000:]

f_bd, Pxx_bd = signal.welch(iq_samples, sample_rate, detrend=lambda x: x)
f_bd, Pxx_bd = fftshift(f_bd), fftshift(Pxx_bd)


plt1.semilogy((f_bd/1e6)+(f_c/1e6), Pxx_bd)
plt1.xlabel('f [MHz]')
plt1.ylabel('PSD [Power/Hz]')
plt1.title('GSM power spectrum')
plt1.grid()
plt1.xticks(np.linspace((-sample_rate/2e6)+(f_c/1e6), (sample_rate/2e6)+(f_c/1e6), 7))
plt1.xlim((-sample_rate/2e6)+(f_c/1e6), (sample_rate/2e6)+(f_c/1e6))
plt1.savefig("GSM_spectrum" + ".png", bbox_inches='tight', pad_inches=0.5)
plt1.show()
#plt1.close()
#print("Processing the samples ...")


In [ ]:


x = range(1,2500)
x = [i/sample_rate/1e-3 for i in x]
f2 = plt2.figure(2)
plt2.plot(x,abs(iq_samples)[1:2500])
plt2.xlabel('Time (ms)')
plt2.title('Magnitude of GSM signal')
plt2.grid()
plt2.show()
#plt2.savefig("GSM_signal" + ".png", bbox_inches='tight', pad_inches=0.5)
#plt2.close()




In [ ]:
t_range, f_range, xmf = myspectrogram_hann_ovlp(iq_samples, 256, sample_rate,f_c,dbf = 60)
spec = abs(xmf)*abs(xmf)
th = .35
z = abs(xmf)*abs(xmf)/(abs(xmf)*abs(xmf)).max()
z[z>th] = 1        

spec = abs(xmf)*abs(xmf)
##print(mean(spec,axis=0))


f4 = plt4.figure(4)
plt4.pcolormesh(t_range, f_range/1e6, 20.0 * np.log10(abs(xmf)))
#plt4.pcolormesh(t_range, f_range/1e6, z)
plt4.xlabel('Time (s)')
plt4.ylabel('Frequency, MHz')
plt4.title('Spectrogram')
plt4.grid()
plt4.show()
#plt4.savefig("GSM_Spectrogram_threshold" + ".png", bbox_inches='tight', pad_inches=0.5)
#plt4.close()




In [ ]:
th = .35
z = abs(xmf)*abs(xmf)/(abs(xmf)*abs(xmf)).max()
z[z>th] = 1        


f4 = plt4.figure(4)
plt4.pcolormesh(t_range, f_range/1e6, z)
plt4.xlabel('Time (s)')
plt4.ylabel('Frequency, MHz')
plt4.title('Spectrogram')
plt4.grid()
plt4.show()
#plt4.savefig("GSM_Spectrogram_threshold" + ".png", bbox_inches='tight', pad_inches=0.5)
#plt4.close()



In [ ]:

trange = np.linspace(1 / sample_rate,len(iq_samples) / sample_rate,len(iq_samples))

f5 = plt5.figure(5)
plt5.plot(trange[1:10000],(iq_samples.real[1:10000]))
##plt4.plot(trange_decimated,decimated.real)
##plt4.plot(trange_filtered,abs(lpfiltered))
##plt4.title('')
plt5.xlabel('Time, s')
plt5.ylabel('Real(s)')
plt5.title('GSM signal after filtering')
plt5.grid()
plt5.show()
#plt5.savefig("GSM_signal_after_filtering" + ".png", bbox_inches='tight', pad_inches=0.5)
#plt5.close()




In [ ]:

numtaps = 140
filter_coeff = signal.firwin(numtaps, .15*sample_rate, nyq=sample_rate, window='nuttall')
lpfiltered = convolve(iq_samples, filter_coeff, mode='same')

dec = 50
decimated = signal.decimate(lpfiltered, dec, zero_phase=True,axis=0)

trange_filtered = np.linspace(1 / sample_rate,len(lpfiltered) / sample_rate,len(lpfiltered))
trange_decimated = np.linspace(dec / sample_rate,dec*len(decimated) / sample_rate,len(decimated))

f5 = plt5.figure(5)
plt5.plot(trange_filtered[1:10000],(lpfiltered.real[1:10000]))
##plt4.plot(trange_decimated,decimated.real)
##plt4.plot(trange_filtered,abs(lpfiltered))
##plt4.title('')
plt5.xlabel('Time, s')
plt5.ylabel('Real(s)')
plt5.title('GSM signal after filtering')
plt5.grid()
plt5.show()
#plt5.savefig("GSM_signal_after_filtering" + ".png", bbox_inches='tight', pad_inches=0.5)
#plt5.close()


In [ ]:

f6 = plt6.figure(6)
# compute average power spectrum
f, sp = avgPS(lpfiltered,N=256,fs=sample_rate/1000)
##fig = figure(figsize=(8,4))
plt6.plot(f,10*log10(sp))
plt6.title('Average power spectrum after filtering')
plt6.xlabel('frequency offset [KHz]')
plt6.grid()
plt6.show()
#plt6.savefig("GSM_power_spectrum_after_filtering" + ".png", bbox_inches='tight', pad_inches=0.5)
#plt6.close()



In [ ]:


### compute average power spectrum
#f, sp = avgPS(decimated,N=64,fs=sample_rate/1000/dec)
#plt7.plot(f,10*log10(sp))
#plt7.title('Average power spectrum')
#plt7.xlabel('frequency offset [KHz]')
#plt7.grid()
#plt7.show()
#plt7.savefig("GSM_power_spectrum" + ".png", bbox_inches='tight', pad_inches=0.5)
#plt7.close()

#print("Processing complete.\n")
#i=0
#detect25 = np.zeros(len(lpfiltered))
#detectLoc = np.zeros(len(lpfiltered))

#for x in lpfiltered:
#    i = i+1

#    if abs(x)>1.5:
#        detect25[i] = 1


#f7 = plt7.figure(7)
#plt7.plot(detect25)
#plt7.xlabel('delay index')
#plt7.ylabel('Peak')
#plt7.grid()
#f7.show()